In this notebook we train our model: a BERT-like model with attention flow inspired by BiDAF.

Dependencies:

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

This part should be data loading and processing.

Input: SQuAD dataset handler/url/json

Output: processed dict/list/whatever: train_question, train_context, train_answer

In [ ]:
from utils import data_processing
""" TODO: moving previous data processing scripts to data_process.py """

This part should be model construction.

In [ ]:
from layers.bert_plus_bidaf import BERT_plus_BiDAF
""" TODO: implement BERT_plus_BiDAF model class """

This part should be declaration of the optimizer and the loss function. 

In [ ]:
""" TODO: optimizer, and loss function(not urgent)

This part should be the definition of training process:

In [ ]:
def train(model, inputs):
""" TODO: determine inputs """

Rest part is for experiments: